## Luftdata med kartor

Simon Winter, Infontology http://infontology.org



I den här lektionen ska vi se hur man kan hitta data för luftföroreningar från det nätverk av sammankopplade sensorer som finns på Luftdata.se

In [1]:
import requests
import pandas as pd
import gmaps
from credentials import gmaps_key, resrobot_key

import gpxpy
import gpxpy.gpx

/Users/management/.local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# https://console.developers.google.com/apis/dashboard?project=natural-furnace-264420
gmaps.configure(api_key=gmaps_key)
HOME = (55.589957, 12.943115)

In [3]:
latitude = '55.611437'
longitude = '12.994264'
distance = '150' # Avstånd i kilometer

Med hjälp av koordinaterna så skapar vi urlen. Lämna tomma platser inom {} i textsträngen, och ge variabelnamnen i format-parentesen

In [4]:
url = 'http://api.luftdaten.info/v1/filter/area={},{},{}'.format(HOME[0], HOME[1], distance)
url ## Om man ger ett variabelnamn på sista raden av en cell skrivs värdet ut. Det går också att göra print(url)

'http://api.luftdaten.info/v1/filter/area=55.589957,12.943115,150'

Requests-biblioteket gör det lätt att hantera frågor till servern. 

In [5]:
response = requests.get(url)

In [ ]:
sensors = response.json()
number_of_sensors = len(sensors)
number_of_sensors

Om number_of_sensors är större än 0 så finns det sensorer i listan. Då kan man få fram sensorvärdena på olika sätt. Det första värdet ligger alltid i sensors[0]

In [ ]:
sens2 = pd.DataFrame(columns=['latitude', 'longitude'])

In [ ]:
sens2

In [ ]:
def create_sensordict (sensor):
    sensordict = {}
    sensordict['longitude'] = sensor['location']['longitude']
    sensordict['latitude'] = sensor['location']['latitude']
    for stype in sensor['sensordatavalues']:
        sensordict[stype['value_type']] = stype['value']
    return sensordict

In [ ]:
for sensor in sensors:
    sens2 = sens2.append (create_sensordict(sensor), ignore_index=True)

    

In [ ]:
sens2

In [ ]:
sens2 = sens2.astype('float')

In [ ]:
sens2.dtypes

In [ ]:
humid_values = sens2[sens2['humidity'].notnull()]

In [ ]:
locations = humid_values[['latitude', 'longitude']]
weights = humid_values['humidity']

In [ ]:
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(point_list)
fig.add_layer(marker_layer)
fig



In [ ]:
value_dict = humid_values.to_dict('index')

In [ ]:
value_dict.values()

In [ ]:
data = humid_values.to_dict('index').values()
#data

In [ ]:

plant_locations = [(plant['latitude'], plant['longitude']) for plant in data] 
info_box_template = """
<dl>
<dt>Fuktighet</dt><dd>{humidity}</dd>
<dt>Lufttryck</dt><dd>{pressure}</dd>
</dl>
"""

plant_info = [info_box_template.format(**plant) for plant in data]
marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)

In [ ]:
fig = gmaps.figure()

fig.add_layer(marker_layer)
fig

In [21]:
# Parsing an existing file:
# -------------------------

with open('min-etapp.gpx') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [22]:
len(gpx.tracks[0].segments[0].points)

1220

In [24]:
point_list = [(point.latitude, point.longitude) for point in gpx.tracks[0].segments[0].points]


In [ ]:
for waypoint in gpx.waypoints:
    print('waypoint {0} -> ({1},{2})'.format(waypoint.name, waypoint.latitude, waypoint.longitude))

In [ ]:
for route in gpx.routes:
    print('Route:')
    for point in route.points:
        print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))

In [ ]:
# There are many more utility methods and functions:
# You can manipulate/add/remove tracks, segments, points, waypoints and routes and
# get the GPX XML file from the resulting object:

print('GPX:', gpx.to_xml())

In [ ]:
fig = gmaps.figure()
geneva2zurich_via_montreux = gmaps.directions_layer(
        point_list[0], HOME, travel_mode='TRANSIT')
point_layer = gmaps.marker_layer([point_list[0], HOME])
fig.add_layer(point_layer)
fig.add_layer(geneva2zurich_via_montreux)
fig

In [25]:
point_list[0]

(56.08004340824433, 13.123206823376563)

In [27]:
point = point_list[0]

In [33]:
url = 'https://api.resrobot.se/v2/location.nearbystops?key={}&originCoordLat={}&originCoordLong={}&format=json'.format(resrobot_key, point[0],point[1]) 

In [34]:
stops = requests.get(url).json()

In [32]:
print(stops)

{}


In [ ]:
sensors[0]

Det finns olika typer av värden:

* P1 är PM10, alltså partiklar mindre än 10 mikrometer
* P2 är PM2.5, alltså partiklar mindre än 2,5 mikrometer
* temperature är temperatur
* humidity är luftfuktighet
* pressure är lufttryck